In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import contextlib2
import io
import IPython
import json
import pathlib
import sys
import tensorflow as tf
import time
import pandas as pd
import numpy as np
import argparse
from PIL import Image, ImageDraw
import os

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
        
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf 
gpus = tf.config.experimental.list_physical_devices('GPU') 
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
def load_model():
    model = tf.compat.v2.saved_model.load(r"../input/resnet50-fpn/resnet50fpn")
#     model = model.signatures['serving_default']

    return model
detect_fn1 =load_model()

In [ ]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

def predict(image):
    return detect_fn1(np.expand_dims(image, axis=0)) 

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()
iter_test = env.iter_test() 

In [ ]:
def format_output(image,pred):
    annot = []
    for i in range(100):
        if pred['detection_scores'][0][i].numpy()>0.25:
            ls=pred['detection_boxes'][0][i].numpy()
            height,width,chal=image_np.shape
            xmin=ls[1]*width
            x2=ls[3]*width
            ymin=ls[0]*height
            y2=ls[2]*height
            height_img=x2-xmin
            width_img=y2-ymin
#             arr={'x':xmin, 'y':ymin, 'width':width_img , 'height':height_img}
            annot.append('{:.2f} {} {} {} {}'.format(pred['detection_scores'][0][i], int(xmin), int(ymin), int(width_img), int(height_img)))
    return annot

In [ ]:
for (image_np, sample_prediction_df) in iter_test:
    pred = predict(image_np)
    predictions=format_output(image_np,pred)
    prediction_str = ' '.join(predictions)
    sample_prediction_df['annotations'] = prediction_str
    env.predict(sample_prediction_df)

In [ ]:
# BASE_DIR = '/kaggle/input/tensorflow-great-barrier-reef'
# testing_data = pd.read_csv(f'{BASE_DIR}/test.csv')
# testing_data.head()
sub=pd.read_csv('./submission.csv')
sub.head()

In [ ]:
# f'{BASE_DIR}/train_images/video_{video_id}/{video_frame}.jpg'
# testing_data.info()
# for i in range(3):
#     print(testing_data.iloc[i])
#     image_path=f'{BASE_DIR}/t_images/video_{video_id}/{video_frame}.jpg'
# import matplotlib.pyplot as plt
# %matplotlib inline
# plt.imshow(ls[0])
# pred=predict(ls[0])